# Рекуррентные нейронные сети

Рекуррентные нейронные сети (RNN) — вид нейронных сетей, где **связи между элементами образуют направленную последовательность**. Благодаря этому появляется возможность обрабатывать серии событий во времени или последовательные пространственные цепочки. В отличие от многослойных перцептронов, рекуррентные сети могут использовать свою **внутреннюю память** для обработки последовательностей произвольной длины.

* [Рекуррентная нейронная сеть - wiki](https://ru.wikipedia.org/wiki/%D0%A0%D0%B5%D0%BA%D1%83%D1%80%D1%80%D0%B5%D0%BD%D1%82%D0%BD%D0%B0%D1%8F_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D0%B0%D1%8F_%D1%81%D0%B5%D1%82%D1%8C)
* [The Unreasonable Effectiveness of Recurrent Neural Networks - Andrej Karpathy blog](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

## Мотивация к применению

Среди основных ограничений DNN и CNN (feedforward networks) можно перечислить следующие:
* На вход принимается **вектор фиксированного размера** и на выходе также создается вектор фиксированного размера;
* Каждый вход подается в нейронную сеть **независимо**, входные элементы не взаимосвязаны.

Но что, если во входные данные представляют из себя что-то целое, но разбитое на части? Текст - последовательность слов, ДНК - последовательность нуклеотидов. Подобные цепочки элементов могут иметь переменную длину.

Для решения задач с такими входными данными была придумана архитектура RNN. В RNN входы зависимы: друг от друга, от своих предыдущих **скрытых** состояний. Они как бы "запоминают" историю всех приходящих к ним на вход данных. Получается, что на этапе обучения нейрон будет принимать на вход свое же значение, вычисленное на предыдущем по "времени" шаге, то есть будет иметь **рекуррентную зависимость**. 

RNN позволяет работать с **последовательностями** векторов с произвольной длиной, позволяя воплотить разные **архитектурные аспекты**:

<img src="pictures/rnn_arc.jpeg" width=600 height=600 />

Благодаря этому можно решать, например, такие задачи, как OCR, машинный перевод, генерация текстов, транскрибация текста и т.д.

## История развития архитектур

| Год | Архитектура | Исследователи | Примечание |
| --- | --- | --- | --- |
| 1980 | Time-delay neural networks | Хинтон | Свертка по времени |
| 1982 | Сеть Хопфилда | Хопфилд |  |
| 1986 | Сеть Джордана | Джордан | SRN (Simple recurrent networks) |
| 1990 | Сеть Элмана | Элман | SRN (Simple recurrent networks) |
| 1997 | [LSTM](https://www.bioinf.jku.at/publications/older/2604.pdf) | Хохрайтер, Шмидхубер |  |
| 2014 | [GRU](https://arxiv.org/pdf/1406.1078.pdf) | Чо, ..., Бенжио |  |
| 2015 | [MUT1](https://proceedings.mlr.press/v37/jozefowicz15.pdf) | ..., Суцкевер |  |
| 2015 | [SCRN (Structurally Constrained Recurrent Neural Network)](https://arxiv.org/pdf/1412.7753v2.pdf) | Миколов | В 4 раза меньше параметров, чем у LSTM |
| 2015 | [uRNN (unitary RNN)](https://arxiv.org/pdf/1511.06464.pdf) | ..., Бенжио |  |
| 2017 | [SRU (Simple Recurrent Unit)](https://arxiv.org/pdf/1709.02755v5.pdf) |  |  |

## Архитектура RNN

Как вычислять градиент функции, если нейрон принимает на вход свое же значение, вычисленное на предыдущем по "времени" шаге? На самом деле, противоречий здесь нет, и в таком графе вычислений не будет никаких циклов. Андрей Карпаты [тонко подмечает](http://karpathy.github.io/2015/05/21/rnn-effectiveness/):

> If training vanilla neural nets is optimization over functions, training recurrent nets is optimization over programs.

Архитектуру RNN можно представить (разворачивание во времени) как последовательные копии одной и той же ячейки, в которую передается некоторая переменная $h$ - **вектор скрытых состояний**. Мы каждый раз объединяем вектор скрытых состояний с предыдущего шага и входной вектор, получая новый вектор скрытых состояний:

<img src="pictures/rnn_forward.png" width=600 height=600 /> 
<img src="pictures/rnn_notation.png" width=600 height=600 />

Vanilla [RNN single cell](https://medium.com/@koushikkushal95/recurrent-neural-networks-part-1-78302d544466) можно представить так, как показано на картинке ниже. Также ниже реализация такой сети на numpy.

<img src="pictures/rnn_cell.jpg" width=500 height=500 />

В вычислениях участвуют постоянно обновляющиеся матрицы $W_{hh}$, $W_{hx}$ и $W_{hy}$. Получается, что на каждом шаге RNN обучает столько итераций, сколько элементов в последовательности - $t = 1, .., T$. Но веса эти на каждом слое одинаковые (shared weights).

<img src="pictures/rnn.png" width=500 height=500 />

**RNN Forward propagation:**
* $h_t = \tanh (W_{hh} h_{t-1} + W_{hx} x_{t})$
* $\hat y_t = \text{softmax} (W_{yh} h_t)$

[**RNN Backward propagation:**](https://mmuratarat.github.io/2019-02-07/bptt-of-rnn)
* $L(\hat y, y) = \sum_{t=1}^T L_t(\hat y_t,y_t)$ - заданная лосс-функция
* Веса матрицы $W_{hh}$ при "разворачивании" графа реккурентной нейронной сети будут участвовать в частных производных backpropagation $T-1$ раз. И никаких циклов.

Основными проблемами RNN явлвются:
* Проблема взрывающегося градиента
* Проблема "влияния" входа или текущего состояния на ответы сети (не затухание, а, скорее, нераспространение): сеть понимает ближайших соседей по буквам или по словам, но "далекие" зависимости, скорее всего, не распознает.

## Глубокие RNN

Vanila RNN - это сеть, в которой используется всего один уровень нейронов. У такой сети могут быть проблемы с обнаружением сложных зависимостей в данных. Тогда мы можем либо усложнять имеющуюся архитектуру, либо предложить что-то новое.

Усложнение архитектуры главным образом связано с моделированием односложных компонент RNN глубокими сетями:
* Можно заменить на многосложную конструкцию функцию от входа к скрытому слою
* Можно заменить на многосложную конструкцию функцию от скрытого слоя к выходу
* Можно заменить на многосложную конструкцию функцию внутри ячейки RNN

> Shortcut connections

Последний вариант предполагает, что мы рассматриваем реккурентную сеть как слой некоторой другой сети, то есть выходы реккурентного слоя будут входами какого-то другого слоя.

### Bidirectional RNN

Bidirectional RNN - это реализация RNN, которая пытается решить проблему "влияния" входа на результат. По сути, это 2 прогона RNN для входной последовательности: слева направо и справо налево. Так мы получаем 2 независимых матрицы скрытых состояний, отражающих контекст для каждого слова и слева, и справа.

<img src="pictures/bidirectional_rnn.jpg" width=500 height=500 />

[picture source](https://colah.github.io/posts/2015-09-NN-Types-FP/?ref=blog.paperspace.com)

## RNN для классификации текстов

In [2]:
import math
import random
import spacy
import string
import sys
import time
import torch
import torch.nn.functional as F
import torchtext

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from torchtext import datasets
from tqdm import tqdm

torch.backends.cudnn.deterministic = True

spacy.load('en_core_web_sm');

In [3]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED);

In [4]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Загрузка данных

In [5]:
df = pd.read_csv('IMDB Dataset.csv')
df['sentiment'] = np.where(df['sentiment'] == 'positive', 1, 0)
df.columns = ['TEXT_COLUMN_NAME', 'LABEL_COLUMN_NAME']
df.shape

(50000, 2)

In [6]:
df.to_csv('moviedata.csv', index=None)
df = pd.read_csv('moviedata.csv')
df.head()

,TEXT_COLUMN_NAME,LABEL_COLUMN_NAME
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


### Обработка данных при помощи torchtext

In [7]:
# Define feature processing (torchtext Field)
TEXT = torchtext.legacy.data.Field(tokenize='spacy',
                                   include_lengths=True,
                                   tokenizer_language='en_core_web_sm')
LABEL = torchtext.legacy.data.LabelField(dtype=torch.float)

/home/kate/anaconda3/lib/python3.8/site-packages/spacy/util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.4.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.3.3). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [8]:
# torchtext TabularDataset
fields = [('TEXT_COLUMN_NAME', TEXT), ('LABEL_COLUMN_NAME', LABEL)]
dataset = torchtext.legacy.data.TabularDataset(path='moviedata.csv',
                                               format='csv',
                                               skip_header=True,
                                               fields=fields)

In [9]:
train_data, test_data = dataset.split(split_ratio=[0.8, 0.2],
                                      random_state=random.seed(RANDOM_SEED))

print('Length of train data', len(train_data))
print('Length of test data', len(test_data))

Length of train data 40000
Length of test data 10000


In [10]:
train_data, val_data = train_data.split(split_ratio=[0.85, 0.15],
                                        random_state=random.seed(RANDOM_SEED))

print('Length of train data', len(train_data))
print('Length of valid data', len(val_data))

Length of train data 34000
Length of valid data 6000


In [11]:
# Build Vocabulary
VOCABULARY_SIZE = 20000
TEXT.build_vocab(train_data,
                 max_size=VOCABULARY_SIZE,
                 vectors='glove.6B.100d',
                 unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 20002
Number of classes: 2


In [12]:
# Define Dataloader
BATCH_SIZE = 128

train_loader, valid_loader, test_loader = torchtext.legacy.data.BucketIterator.splits(
    (train_data, val_data, test_data), 
    batch_size=BATCH_SIZE,
    sort_key=lambda x: len(x.TEXT_COLUMN_NAME),
    sort_within_batch=True,
    device=DEVICE)

In [13]:
LEARNING_RATE = 1e-4
NUM_EPOCHS = 15

### Реализация Vanilla RNN

[Inspiration code source](https://github.com/hrishikeshshekhar/Vanilla-RNN/tree/master)

In [14]:
class VanillaRNN:
    def __init__(self,
                 input_dim,
                 output_dim,
                 sentence_length, 
                 hidden_dim=64,
                 learning_rate=0.001,
                 momentum=0.9):

        self.minibatches = 1
        self.beta = momentum
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.learning_rate = learning_rate
        self.sentence_length = sentence_length

        # Calculating initializer constants (xavier)
        whx = math.sqrt(6) / math.sqrt(self.hidden_dim + self.input_dim)
        whh = math.sqrt(6) / math.sqrt(self.hidden_dim + self.hidden_dim)
        wyh = math.sqrt(6) / math.sqrt(self.output_dim + self.hidden_dim)

        # Initial weights
        self.Whx = np.random.uniform(-whx, whx, (hidden_dim, input_dim))
        self.Whh = np.random.uniform(-whh, whh, (hidden_dim, hidden_dim))
        self.Wyh = np.random.uniform(-wyh, wyh, (output_dim, hidden_dim))
        
        # normal distribution
        # self.Whx = np.random.normal(0, 1, (hidden_dim, input_dim))
        # self.Whh = np.random.normal(0, 1, (hidden_dim, hidden_dim))
        # self.Wyh = np.random.normal(0, 1, (output_dim, hidden_dim))

        # Initial biases
        self.bh = np.random.normal(0, 1, (hidden_dim, 1))
        self.by = np.random.normal(0, 1, (output_dim, 1))

        # Momentum optimizer variables
        self.dWhx = np.zeros((hidden_dim, input_dim))
        self.dWhh = np.zeros((hidden_dim, hidden_dim))
        self.dWyh = np.zeros((output_dim, hidden_dim))
        self.dbh = np.zeros((hidden_dim, 1))
        self.dby  = np.zeros((output_dim, 1))

        # Count total trainable parameters
        total_params = (
            (self.hidden_dim * self.hidden_dim)
            + (self.input_dim * self.hidden_dim)
            + (self.output_dim * self.hidden_dim)
            + (self.hidden_dim)
            + (self.output_dim)
        )
        print(f"Total trainable parameters: {total_params}")

    def forward(self, data):
        # Check input dimensions
        assert(data.shape[0] == self.sentence_length)
        assert(data.shape[1] == self.input_dim)

        # Initializing initial state as a zero vector
        batch_size = data.shape[2]
        h = np.zeros((self.hidden_dim, batch_size))

        # Calculating states
        states = [h]
        for words in data:
            assert(np.array(words).shape == (self.input_dim, batch_size))
            h = np.tanh(np.matmul(self.Whh, h) + np.matmul(self.Whx, words) + self.bh)
            states.append(h)

        # [sentence_length + 1, self.hidden_dim, batch_size]
        states = np.array(states)

        # Calculating output: [self.output_dim, batch_size]
        output = np.matmul(self.Wyh, states[-1]) + self.by
        output = np.array(self.softmax(output)).reshape((self.output_dim, batch_size))

        return output, states

    def softmax(self, data):
        np.clip(data, -200, 200, out=data)
        data = np.exp(data)
        data /= np.sum(data, axis=0)
        return data

    def backward(self, train_X, train_Y, preds, states, batch_size):
        # [self.output_dim, batch_size]
        dL_dY = preds.T

        for index, pred in enumerate(dL_dY):
            pred[train_Y[index]] -= 1

        dL_dY = dL_dY.T

        # Timesteps
        T = len(states) - 1

        # Calculating gradients for Wyh and by
        # Shape (self.output_dim, self.hidden_dim)
        dL_dWyh = np.matmul(dL_dY, states[-1].T)
        dL_dby = np.sum(dL_dY, axis=1).reshape(self.output_dim, 1)

        # Calculating gradients for Whx, Whh, bh
        dL_dWhh = np.zeros(shape=self.Whh.shape)
        dL_dWhx = np.zeros(shape=self.Whx.shape)
        dL_dbh = np.zeros(shape=self.bh.shape)

        # Calculating dL / dh
        # Shape(self.hidden_dim, batch_size)
        dL_dh = np.matmul(self.Wyh.T, dL_dY)

        for t in reversed(range(T)):
            dL_dWhh += np.matmul(dL_dh * (1 - (states[t + 1] ** 2)), states[t].T)
            dL_dWhx += np.matmul(dL_dh * (1 - (states[t + 1] ** 2)), train_X[t].T)

            dL_dbh += np.sum(dL_dh * (1 - (states[t + 1] ** 2)), axis=1).reshape(self.hidden_dim, 1)

            # Updating dL_dh
            dL_dh = np.matmul(self.Whh, dL_dh * (1 - states[t + 1] ** 2))
            
        # Applying momentum and normalizing
        # Calculating exponential averages
        normalization_factor = 1 - (self.beta ** min(self.minibatches, 100))
        self.dWhh = (self.beta * self.dWhh + (1 - self.beta) * (dL_dWhh)) / normalization_factor
        self.dWhx = (self.beta * self.dWhx + (1 - self.beta) * (dL_dWhx)) / normalization_factor
        self.dWyh = (self.beta * self.dWyh + (1 - self.beta) * (dL_dWyh)) / normalization_factor
        self.dbh  = (self.beta * self.dbh + (1 - self.beta) * (dL_dbh)) / normalization_factor
        self.dby  = (self.beta * self.dby + (1 - self.beta) * (dL_dby)) / normalization_factor

        # Clipping the gradients for exploding gradients
        for updates in [self.dWhh, self.dWhx, self.dWyh, self.dbh, self.dby]:
            np.clip(updates, -1, 1, out=updates)

        # Updating the weights and biases
        self.Whh -= self.learning_rate * self.dWhh
        self.Whx -= self.learning_rate * self.dWhx
        self.Wyh -= self.learning_rate * self.dWyh
        self.bh  -= self.learning_rate * self.dbh
        self.by  -= self.learning_rate * self.dby

    def train(self, train_X, train_Y, test_X, test_Y, epochs, batch_size=32):
        # Checking train
        train_X = np.array(train_X)
        train_Y = [int(target) for target in train_Y]
        training_size = train_X.shape[0]
        assert(training_size == len(train_Y))
        assert(train_X.shape[1] == self.sentence_length)
        assert(train_X.shape[2] == self.input_dim)
        
        # Checking test
        test_X = np.array(test_X)
        test_Y = [int(target) for target in test_Y]
        testing_size = test_X.shape[0]
        assert(testing_size == len(test_Y))
        assert(test_X.shape[1] == self.sentence_length)
        assert(test_X.shape[2] == self.input_dim)

        # Conforming batch size to a maximum of training_size / 2
        batch_size = min(batch_size, training_size / 2)

        # Transposing the testing data
        test_X = np.transpose(test_X, (1, 2, 0))

        # Array to store metrics
        losses = []
        correct_ans = []
        log_frequency = max(int(float(epochs) / 100), 1)

        # Splitting the training data into batches of size batchsize
        batches = int(math.ceil(float(training_size) / batch_size))
        batch_training_X = []
        batch_training_Y = []

        # Splitting training data into batches
        for batch in range(batches):
            start_index = batch_size * batch
            end_index = min(batch_size * (batch + 1), training_size)
            temp_batch_size = end_index - start_index
            batch_train_X = train_X[start_index: end_index]

            # Creating an np array of size (batch_size, max_batch_length, self.hidden_dim)
            batch_train_X = np.array(batch_train_X).reshape(
                (temp_batch_size, self.sentence_length, self.input_dim))
            batch_train_Y = np.array(train_Y[start_index: end_index])
            batch_training_X.append(batch_train_X)
            batch_training_Y.append(batch_train_Y)

        # Checking if the correct number of batches were inserted
        assert(len(batch_training_X) == batches)

        # Deleting variables to clear RAM
        del train_X, train_Y

        # Training the net
        for epoch in range(epochs):
            loss = 0
            num_correct = 0

            # Iterating through each training batch
            for batch in range(batches):
                # Picking out one batch of training samples
                train_X = batch_training_X[batch]
                train_Y = batch_training_Y[batch]

                # Train_X from (batch_size, sentence_length, self.input_dim) 
                # to (sentence_length, self.input_dim, batch_size)
                train_X = np.transpose(train_X, (1, 2, 0))

                # Feed Forwarding
                preds, states = self.forward(train_X)

                loss -= np.sum(np.log([pred[train_Y[index]] for index, pred in enumerate(preds.T)]))
                num_correct += np.sum(np.argmax(preds, axis=0) == train_Y)

                # Back propagating the error
                self.backward(train_X, train_Y, preds, states, batch_size)

                # Updating the mini batch number
                self.minibatches += 1

            # Appending loss to training data
            losses.append(loss)
            correct_ans.append((float(num_correct) / training_size) * 100)

            # Printing loss and number of correctly classified values
            if(epoch % log_frequency == 0):
                train_loss = round(loss / training_size, 3)
                train_accuracy = round(float(num_correct) / training_size, 3)

                # Resetting loss and correct answers
                loss = 0
                num_correct = 0

                # Feed Forwarding
                preds, states = self.forward(test_X)

                # Calculating the loss and correct classifications
                loss -= np.sum(np.log([pred[test_Y[index]] for index, pred in enumerate(preds.T)]))
                num_correct += np.sum(np.argmax(preds, axis=0) == test_Y)
                
                test_loss = round(loss / testing_size, 3)
                test_accuracy = round(float(num_correct) / testing_size, 3)
                
                print(
                    f"Epoch: {epoch+1:03d}/{epochs:03d} | "
                    f"train loss: {train_loss:.3f} | test loss: {test_loss:.3f} | "
                    f"train acc: {train_accuracy:.3f} | test acc: {test_accuracy:.3f}"
                )

        return losses, correct_ans

In [15]:
SENTENCE_LENGTH = 50
EMBEDDING_DIM = 50
HIDDEN_DIM = 128
OUTPUT_DIM = 2

rnn = VanillaRNN(EMBEDDING_DIM,
                 OUTPUT_DIM,
                 SENTENCE_LENGTH,
                 hidden_dim=HIDDEN_DIM,
                 learning_rate=LEARNING_RATE)

Total trainable parameters: 23170


In [16]:
import string
import numpy as np
import io
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


class GloveEmbeddings:
    def __init__(self, sentence_length, embedding_dim=50, remove_stop_words=False):
        self.sentence_length = sentence_length
        self.embedding_dim = embedding_dim
        self.remove_stop_words = remove_stop_words

        glove_path = "/home/kate/Desktop/projects/ML_DL_experiments/nn/glove.6B.50d.txt"
        self.word2vec = self.load_glove_model(glove_path)
        self.stopwords = set(stopwords.words('english'))
        self.outlier_words = []

    def load_glove_model(self, gloveFile):
        f = io.open(gloveFile, 'r', encoding="utf-8")
        model = {}
        for line in f:
            splitLine = line.split()
            word = splitLine[0]
            embedding = np.array([float(val) for val in splitLine[1:]])
            model[word] = embedding
        print(f"{len(model)} words loaded!")
        return model

    def remove_punctuation(self, sentence):
        output = ""
        for symbol in sentence:
            # Replacing all special symbols with a space 
            if (symbol in string.punctuation):
                output += " "
            else:
                output +=  symbol
        return output

    def tokenize(self, input_sentence):
        sentence = self.remove_punctuation(input_sentence)
        words = word_tokenize(sentence)
        words = [word.lower() for word in words]

        if self.remove_stop_words:
            new_words = []
            for word in words:
                if(word not in self.stopwords):
                    new_words.append(word)
            return new_words
        else:
            return words
        
    def pad_sentence(self, words):
        padding = [0 for _ in range(self.embedding_dim)]
        sentence_length = len(words)
        if(sentence_length < self.sentence_length):
            for _ in range(self.sentence_length - sentence_length):
                words.append(padding)
        return words

    def trim_sentence(self, words):
        if(len(words) > self.sentence_length):
            return words[:self.sentence_length]
        return words

    def create_input(self, sentence):
        words = self.tokenize(sentence)
        inputs = []

        for index, word in enumerate(words):
            try:
                inputs.append(self.word2vec[word])
            except:
                self.outlier_words.append(word)
                inputs.append(np.zeros(self.embedding_dim))

        padded_inputs = self.pad_sentence(inputs)
        trimmed_inputs = self.trim_sentence(padded_inputs)
        inputs = np.array(trimmed_inputs).reshape(
            self.sentence_length, self.embedding_dim)
        return inputs

    def get_data_from_list(self, sentences):
        data_size = len(sentences)
        output_data = np.array([self.create_input(sentence) for sentence in sentences])
        assert(output_data.shape == (data_size, self.sentence_length, self.embedding_dim))
        return np.array(output_data)

In [17]:
# Creating an embeddings class object
embedding = GloveEmbeddings(SENTENCE_LENGTH,
                            embedding_dim=EMBEDDING_DIM,
                            remove_stop_words=True)

400000 words loaded!


In [18]:
X, y = df['TEXT_COLUMN_NAME'], df['LABEL_COLUMN_NAME']
train_X, test_X, train_Y, test_Y = train_test_split(X, y, test_size=0.33, random_state=42)

In [19]:
# Perparing the input data
train_X = embedding.get_data_from_list(train_X)
test_X = embedding.get_data_from_list(test_X)

In [20]:
# Training RNN
losses, correct_values = rnn.train(train_X, train_Y,
                                   test_X, test_Y,
                                   NUM_EPOCHS,
                                   batch_size=BATCH_SIZE)

Epoch: 001/015 | train loss: 0.725 | test loss: 0.702 | train acc: 0.505 | test acc: 0.530
Epoch: 002/015 | train loss: 0.696 | test loss: 0.690 | train acc: 0.533 | test acc: 0.550
Epoch: 003/015 | train loss: 0.688 | test loss: 0.684 | train acc: 0.551 | test acc: 0.566
Epoch: 004/015 | train loss: 0.682 | test loss: 0.679 | train acc: 0.564 | test acc: 0.574
Epoch: 005/015 | train loss: 0.676 | test loss: 0.672 | train acc: 0.576 | test acc: 0.588
Epoch: 006/015 | train loss: 0.662 | test loss: 0.631 | train acc: 0.607 | test acc: 0.657
Epoch: 007/015 | train loss: 0.620 | test loss: 0.615 | train acc: 0.668 | test acc: 0.680
Epoch: 008/015 | train loss: 0.609 | test loss: 0.606 | train acc: 0.682 | test acc: 0.689
Epoch: 009/015 | train loss: 0.598 | test loss: 0.605 | train acc: 0.694 | test acc: 0.688
Epoch: 010/015 | train loss: 0.596 | test loss: 0.600 | train acc: 0.695 | test acc: 0.691
Epoch: 011/015 | train loss: 0.594 | test loss: 0.602 | train acc: 0.696 | test acc: 0.687

### Реализация Pytorch RNN

[Inspiration code source](https://github.com/rasbt/deeplearning-models/blob/master/pytorch_ipynb/rnn/rnn_lstm_packed_imdb-glove.ipynb)

In [9]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)

        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        # self.rnn = nn.LSTM(embedding_dim, hidden_dim)

        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_length):
        embedded = self.embedding(text)
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, text_length)
        
        packed_output, hidden = self.rnn(packed)
        # packed_output, (hidden, cell) = self.rnn(packed)
        
        return self.fc(hidden.squeeze(0)).view(-1)

In [10]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 128
HIDDEN_DIM = 256
OUTPUT_DIM = 1

torch.manual_seed(RANDOM_SEED)
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [11]:
def compute_binary_accuracy(model, data_loader, device):
    model.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(data_loader):
            text, text_lengths = batch_data.TEXT_COLUMN_NAME
            logits = model(text, text_lengths)
            predicted_labels = (torch.sigmoid(logits) > 0.5).long()
            num_examples += batch_data.LABEL_COLUMN_NAME.size(0)
            correct_pred += (predicted_labels == batch_data.LABEL_COLUMN_NAME.long()).sum()
        return correct_pred.float()/num_examples

In [12]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(tqdm(train_loader)):
        text, text_lengths = batch_data.TEXT_COLUMN_NAME
        label = batch_data.LABEL_COLUMN_NAME
        
        ### FORWARD AND BACK PROP
        logits = model(text, text_lengths)
        train_loss = F.binary_cross_entropy_with_logits(logits, label)
        optimizer.zero_grad()
        
        train_loss.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()

    ### VALIDATION
    for batch_idx, batch_data in enumerate(valid_loader):
        text, text_lengths = batch_data.TEXT_COLUMN_NAME
        label = batch_data.LABEL_COLUMN_NAME
        logits = model(text, text_lengths)
        valid_loss = F.binary_cross_entropy_with_logits(logits, label)

    ### LOGGING
    with torch.set_grad_enabled(False):
        train_acc = compute_binary_accuracy(model, train_loader, DEVICE)
        valid_acc = compute_binary_accuracy(model, valid_loader, DEVICE)
    
    print(
        f'{epoch+1:03d}/{NUM_EPOCHS:03d} | '
        f'Train loss: {train_loss:.3f} | Valid loss: {valid_loss:.3f} | '
        f'Train acc: {train_acc:.3f} | Valid acc: {valid_acc:.3f} | '
        f'{(time.time() - start_time)/60:.2f} min'
    )

print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_binary_accuracy(model, test_loader, DEVICE):.2f}')

100%|██████████| 266/266 [39:04<00:00,  8.81s/it]  


001/015 | Train loss: 0.620 | Valid loss: 0.655 | Train acc: 0.659 | Valid acc: 0.648 | 40.04 min


100%|██████████| 266/266 [33:40<00:00,  7.60s/it]  


002/015 | Train loss: 0.662 | Valid loss: 0.672 | Train acc: 0.651 | Valid acc: 0.637 | 74.68 min


100%|██████████| 266/266 [34:29<00:00,  7.78s/it]  


...

013/015 | Train loss: 0.660 | Valid loss: 0.667 | Train acc: 0.546 | Valid acc: 0.537 | 464.10 min


100%|██████████| 266/266 [38:26<00:00,  8.67s/it]  


014/015 | Train loss: 0.470 | Valid loss: 0.616 | Train acc: 0.775 | Valid acc: 0.750 | 503.49 min


100%|██████████| 266/266 [38:04<00:00,  8.59s/it]  


015/015 | Train loss: 0.425 | Valid loss: 0.565 | Train acc: 0.785 | Valid acc: 0.752 | 542.96 min
Total Training Time: 542.96 min
Test accuracy: 0.75


## Архитектура LSTM (Long Short Term Memory)

* [Understanding LSTM Networks - colah's blog](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Long Short Term Memory (LSTM) – подкласс рекуррентных нейронных сетей, которые были предложены в 1997 Хохрайтером и Шмидхубером. Они гораздо лучше классических RNN справляются с "запоминанием" долгосрочных зависимостей.

Архитектура LSTM представляет из себя усложнение конструкции внутри ячейки RNN: вместо одного слоя в элементе содержится 4 слоя с разными функциями (желтые квадраты). Также каждая LSTM-ячейка имеет **состояние**, на которые влияют три вида узлов, называемых **гейтами**: входной (input gate), забывающий (forget gate) и выходной (output gate). И еще каждая ячейка имеет 3 входа: входной вектор $x_t$, состояние ячейки $c_t$ и скрытое состояние $h_t$ (во время $t$).

<img src="pictures/lstm.png" width=600 height=600 />

Вычисления внутри LSTM-ячейки: для входа $x_t$ и значения вектора скрытого состояния на предыдущем шаге $h_{t-1}$ и вектора собственного состояния яейки на предыдущем шаге $c_{t-1}$:
* $c_t' = \text{tanh}(W_{xc} x_t + W_{hc} h_{t-1} + b_{c'})$ - candidate cell state
* $i_t = \sigma (W_{xi} x_t + W_{hi} h_{t-1} + b_{i})$ - input gate
* $f_t = \sigma (W_{xf} x_t + W_{hf} h_{t-1} + b_{f})$ - forgate gate
* $o_t = \sigma (W_{xo} x_t + W_{ho} h_{t-1} + b_{o})$ - output gate
* $c_t = f_t \cdot c_{t-1} + i_t \cdot c_t'$ - cell state
* $h_t = o_t \cdot \text{tanh}(c_t)$ - block output

Разные слои LSTM отвечают за разные задачи:
* Один слой вычисляет, насколько на данном шаге ему нужно забыть предыдущую информацию 
* Другой слой вычисляет, насколько ему интересна новая информация, пришедшая с сигналом
* На третьем слое вычисляется линейная комбинация памяти и наблюдения с только вычисленными весами для каждой из компонент. Так получается новое состояние памяти, которое в таком же виде передаётся далее.

**Основная идея LSTM**
* Состояние ячейки — горизонтальная линия, проходящая через верхнюю часть диаграммы
* При обучении нейронной сети параметром является размер ячейки
* 3 гейта регулируют то, какую часть информации в состоянии ячейки нужно забыть, а какую запомнить
* Сигмоида принимает значения от 0 до 1, описывающие, сколько каждого компонента должно быть пропущено: 0 - это ничего не пропускать, 1 - забыть все

<img src="pictures/lstm1.png" width=600 height=600 />

**Первый шаг LSTM**
* Первый слой вычисляет, насколько на данном шаге ему нужно забыть предыдущую информацию, какую информацию можно выбросить из ячейки (forget gate layer):
* Результат будет перемножен со значениями ячейки. Некоторые значения могут обнулиться  или снизить свой вклад

<img src="pictures/lstm2.png" width=600 height=600 />

**Внесение новой информации в ячейку**
* Второй слой вычисляет, насколько ему интересна новая информация, пришедшая с сигналом.
* Этап состоит из двух частей:
    1. Сначала сигмоидальный слой (input layer gate) определяет, какие значения следует обновить
    2. Затем tanh-слой строит вектор новых значений-кандидатов, которые можно добавить в состояние ячейки

<img src="pictures/lstm3.png" width=600 height=600 />

**Обновление ячейки памяти**
* На третьем слое вычисляется линейная комбинация памяти и наблюдения с только вычисленными весами для каждой из компонент:
    * Старое нужно забыть в соответствии с $f_t$ 
    * Прибавляем новые значения, умноженные на коэффициент обновления $i_t$

<img src="pictures/lstm4.png" width=600 height=600 />

**Выработка выходного значения (выходной слой:)**
* Выходные данные основываются на состоянии ячейки
* Применяется еще один фильтр (сигмоида), чтобы решить какую информацию нужно использовать

<img src="pictures/lstm5.png" width=600 height=600 />

### Модификации LSTM

**Peephole connections**

<img src="pictures/peephole_lstm.png" width=600 height=600 />

**Coupled forget and input gates**

<img src="pictures/lstm_mod2.png" width=600 height=600 />

### Достоинства LSTM

Все типы рекуррентных нейронных сетей (RNN, LSTM и GRU) страдают от того, что последние элементы последовательности влияют на результат больше, чем первые. Однако, LSTM гораздо лучше справляются с проблемой исчезающих градиентов, чем обычные RNN.

> Constatnt error carousel (рекурсивное вычисление без нелинейности)

### Рекомендации по настройке параметров LSTM
* Использовать gradient clipping (clipnorm или clipvalue) для предотвращения проблемы взрывающихся градиентов
* Использовать нормализацию по батчам и инициализацию весов Xavier
* Инициализировать свободный член $b_f$ большими случайными числами (от 1 до 2), чтобы constatnt error carousel "работала"
* Использовать разные типы регуляризации: L1, L2, dropout 
* Ограничивать количество параметров для небольших выборок
* Использовать функцию активации softsign (не softmax) через tanh (она быстрее и менее подвержена насыщению
* Использовать RMSProp, Adam, AdaGrad или Nesterov momentum, Adagrad иногда тоже неплохой выбор

## Архитектура GRU

GRU - вариант LSTM со значительно упрощенной архитектурой: в классической архитектуре LSTM целых 8 матриц весов + еще 3 матрицы, в случае использования peephole. Если же мы будем использовать модификацию LSTM, в которой "свяжем" входной и забывающий гейты, то получим архитектуру GRU: gated recurrent union.

<img src="pictures/gru.png" width=600 height=600 />